# Rotation and Arias Intensity

In [1]:
import glob
from amptools.io.read import read_data
from amptools.stream import group_channels
import numpy as np
from scipy import integrate
import scipy.constants as sp
import matplotlib.pyplot as plt


In [2]:
def get_arias_intensity(acc, dt, starttime):
    """Calculates Arias Intensity.

    Args:
        acc (array): Array of acceleration values in m/s/s.
        dt (float): Time between each sample in s.
        starttime (float): Time in s, after start of record, to start
            integrating. Usually P-wave arrival time or 0 for full record.

    Returns:
        Ia (array): Array of Arias intensity values in m/s with respect to
            time.
        NIa (array): Array of normalized Arias intensity values with respect to
            time. 
    """

    g = sp.g
    npts = len(acc)
    t = np.linspace(0, (npts-1)*dt, num=npts)
    
    # Choose acceleration values starting at the specificed starttime
    acc2 = acc[t >= starttime]
    
    # Calculate Arias intensity 
    Int = integrate.cumtrapz(acc2*acc2, dx=dt)
    Ia = Int * np.pi/(2*g)

    # Calculate normalized Arias intensity
    # divide arias intensity by its max value
    NIa = Ia/np.amax(Ia)
    return(Ia, NIa)

def plot_arias_intensity(Ia, dt):
    """Plots Arias intensity.

    Args:
        Ia (array): Array of Arias intensity values with respect to time.
        dt (float): time in between each record in s.

    Returns:
    """

    npts = len(Ia)
    t = np.linspace(0, (npts-1)*dt, num=npts)

    # Plot Arias Intensity
    plt.plot(t, Ia, 'k-')
    plt.xlabel('Time (s)')
    plt.ylabel('Arias Intensity (m/s)')
    plt.savefig('Arias_Intensity.png')

def gmrotd(osc1, osc2, percentiles):
    osc1_rot, osc2_rot = rotate(osc1, osc2, combine=False)
    osc1_max = np.amax(osc1_rot, 1)
    osc2_max = np.amax(osc2_rot, 1)
    GMs = np.sqrt(osc1_max * osc2_max)
    return GMs, np.percentile(GMs, percentiles)

def rotd(osc1, osc2, percentiles):
    rot = rotate(osc1, osc2, True)
    maximums = np.amax(np.abs(rot), 1)
    return maximums, np.percentile(maximums, percentiles)

def rotate(osc1, osc2, combine=False):
    if combine:
        degrees = np.deg2rad(np.linspace(0, 180, 181))
        shape = (len(osc1), 1)
        degree_matrix = np.multiply(np.ones(shape), degrees).T
        cos_matrix = np.cos(degree_matrix)
        sin_matrix = np.sin(degree_matrix)

        # Create timeseries matrix
        osc1_matrix = np.multiply(np.ones((181, 1)), osc1)
        osc2_matrix = np.multiply(np.ones((181, 1)), osc2)

        # Calculate GMs
        rot = osc1_matrix * cos_matrix + osc2_matrix * sin_matrix
        return rot
    else:
        degrees = np.deg2rad(np.linspace(0, 90, 91))
        shape = (len(osc1), 1)
        degree_matrix = np.multiply(np.ones(shape), degrees).T
        cos_matrix = np.cos(degree_matrix)
        sin_matrix = np.sin(degree_matrix)

        # Create timeseries matrix
        osc1_matrix = np.multiply(np.ones((91, 1)), osc1)
        osc2_matrix = np.multiply(np.ones((91, 1)), osc2)

        # Calculate GMs with rotation
        osc1_rot = osc1_matrix * cos_matrix + osc2_matrix * sin_matrix
        osc2_rot = osc1_matrix * sin_matrix + osc2_matrix * cos_matrix
        return osc1_rot, osc2_rot
        

In [3]:
streams = []
for file_path in glob.glob('tests/data/knet/AOM001*'):
    streams += [read_data(file_path)]
stream = group_channels(streams)[0]

In [4]:
# Calculate Arias and then rotate
arias_dict = {}
for trace in stream:
    channel = trace.stats['channel']
    Ia, NIa = get_arias_intensity(trace.data.copy(), trace.stats['delta'], 0)
    arias = {'Ia': Ia, 'NIa': NIa, 'dt': trace.stats['delta']}
    arias_dict[channel] = arias
# Get GMRot
osc1 = arias_dict['HN1']['Ia']
osc2 = arias_dict['HN2']['Ia']
# Get geometric means at each angle and gmrotd 0, 50, 100 
GM1, gmrotd1 = gmrotd(osc1, osc2, [0, 50, 100])
# Get max at each angle and rotd 0, 50, 100 
maximum1, rotd1 = rotd(osc1, osc2, [0, 50, 100])

In [8]:
# Rotate and then take arias intensity
osc1 = stream.select(channel="HN1")[0]
osc2 = stream.select(channel="HN2")[0]

# rotate two 
osc1_rot, osc2_rot = rotate(osc1, osc2, combine=False)

# take arias intensity
counter = 0
for angle1, angle2 in zip(osc1_rot, osc2_rot):
    Ia1, NIA1 = get_arias_intensity(angle1, osc1.stats['delta'], 0)
    Ia2, NIA2 = get_arias_intensity(angle2, osc2.stats['delta'], 0)
    if counter == 0:
        arias_matrix1 = [Ia1]
        arias_matrix2 = [Ia2]
    else:
        arias_matrix1 = np.append(arias_matrix1, [Ia1], axis=0)
        arias_matrix2 = np.append(arias_matrix2, [Ia2], axis=0)
    counter += 1
arias1_max = np.amax(arias_matrix1, 1)
arias2_max = np.amax(arias_matrix2, 1)
GM2 = np.sqrt(arias1_max * arias2_max)
gmrotd2 = np.percentile(GM2, [0, 50, 100])

print('GEOMETRIC MEANS:', GM1==GM2)
print('GMROTD', gmrotd1, gmrotd2, gmrotd1==gmrotd2)

# rotate one 
rot = rotate(osc1, osc2, combine=True)

# take arias intensity
counter = 0
for angle in zip(rot):
    Ia, NIA = get_arias_intensity(angle1, osc1.stats['delta'], 0)
    if counter == 0:
        arias_matrix = [Ia]
    else:
        arias_matrix = np.append(arias_matrix, [Ia], axis=0)
    counter += 1
maximum2 = np.amax(np.abs(arias_matrix), 1)
rotd2 = np.percentile(maximum2, [0, 50, 100])
print('MAXIMUMS:', maximum1==maximum2)
print('ROTD', rotd1, rotd2, rotd1==rotd2)

GEOMETRIC MEANS: [ True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False  True]
GMROTD [  8.29260564  10.80368022  11.73870557] [ 7.80329689  7.950848    8.29260564] [False False False]
MAXIMUMS: [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False F